In [5]:
%pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357250 sha256=23518555fc8f313e00253bedcb1976153c1f0190fec60599a3d6bd5ee88b9608
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [6]:
import surprise

In [7]:
data = surprise.Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [9]:
data.raw_ratings[:10]

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [11]:
#  user 열은 사용자 아이디, item 열은 상품 아이디, rate 열은 평점이다.
import pandas as pd
df = pd.DataFrame(data.raw_ratings, columns=["user", "item", "rate", "id"])
del df["id"]
df.head()

,user,item,rate
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0


추천 시스템은 사용자 아이디와 상품 아이디라는 두 개의 카테고리 입력과 평점 출력을 가지는 예측 시스템

In [12]:
# x축이 상품, y축이 사용자 아이디인 평점 행렬(rate matrix) R
df_table = df.set_index(["user", "item"]).unstack()
df_table.shape

(943, 1682)

In [13]:
# 평점 행렬의 일부만 살펴보면 다음과 같이 평점 데이터가 일부 위치에만 존재하는 sparse 행렬임을 알 수 있다.
df_table.iloc[212:222, 808:817].fillna("")

rate                                        
item  211  212  213  214  215  216  217  218  219
user                                             
290   3.0                      4.0       2.0     
291        4.0       4.0  4.0            4.0  4.0
292                  3.0                         
293   4.0       3.0       4.0  4.0  3.0  2.0     
294                                              
295             5.0       5.0  5.0  4.0  5.0     
296   4.0                                        
297   4.0       3.0       2.0  4.0       3.0     
298   5.0       3.0       5.0                    
299   4.0  4.0  5.0            5.0

MovieLens 100k 데이터셋 요약

데이터셋 설명
- 출처: MovieLens, 영화 추천 서비스
- 내용: 5점 만점의 평점과 자유 텍스트 태깅 활동
- 범위: 1996년 3월 29일부터 2018년 9월 24일까지
- 규모: 100,836개의 평점과 3,683개의 태그 적용
- 영화 수: 9,742편
- 사용자 수: 610명
- 생성일: 2018년 9월 26일

파일 구성
- ratings.csv: 영화 평점 정보 (userId, movieId, rating, timestamp)
- tags.csv: 사용자 태그 정보 (userId, movieId, tag, timestamp)
- movies.csv: 영화 정보 (movieId, title, genres)
- links.csv: 다른 데이터 소스로의 링크 (movieId, imdbId, tmdbId)


In [17]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
from google.colab import drive

drive.mount('/content/drive')

# CSV 파일 경로 설정
ratings_file_path = '/content/drive/MyDrive/kdt_240424/m5_머신러닝/dataset/ml-latest-small/ratings.csv'

# 데이터 로드
ratings_df = pd.read_csv(ratings_file_path)
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

# 학습 및 테스트 데이터셋 분리
trainset, testset = train_test_split(data, test_size=0.25)

# SVD 알고리즘 사용하여 모델 학습
algo = SVD()
algo.fit(trainset)

# 테스트 데이터셋 예측
predictions = algo.test(testset)

# 정확도 계산
accuracy.rmse(predictions)

# 학습 데이터셋에 있는 사용자 ID 확인
trainset_user_ids = [trainset.to_raw_uid(i) for i in range(trainset.n_users)]
print("Trainset 사용자 ID 예시:", trainset_user_ids[:10])

# 학습 데이터셋에 있는 사용자 중 하나를 선택
user_id = trainset_user_ids[0]  # 첫 번째 사용자 ID를 선택

user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
items_to_predict = [trainset.to_raw_iid(item) for item, rating in user_ratings]

# 예측되지 않은 아이템 리스트
all_items = set(trainset.all_items())
items_to_predict = list(all_items - set(items_to_predict))

# 아이템 예측
predictions = [algo.predict(user_id, item) for item in items_to_predict]

# 평점이 높은 순으로 추천
predictions.sort(key=lambda x: x.est, reverse=True)
top_predictions = predictions[:10]

print("Top 10 추천 영화:")
for prediction in top_predictions:
    print(f"Movie ID: {prediction.iid}, Predicted Rating: {prediction.est:.2f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
RMSE: 0.8760
Trainset 사용자 ID 예시: [489, 474, 69, 201, 502, 594, 165, 448, 113, 275]
Top 10 추천 영화:
Movie ID: 527, Predicted Rating: 4.61
Movie ID: 1945, Predicted Rating: 4.34
Movie ID: 1223, Predicted Rating: 4.28
Movie ID: 541, Predicted Rating: 4.20
Movie ID: 318, Predicted Rating: 4.20
Movie ID: 2019, Predicted Rating: 4.18
Movie ID: 50, Predicted Rating: 4.17
Movie ID: 4973, Predicted Rating: 4.17
Movie ID: 1248, Predicted Rating: 4.17
Movie ID: 1265, Predicted Rating: 4.14


과제. Surprise 라이브러리를 사용하여 MovieLens 데이터셋을 기반으로 추천 시스템을 구축하고, 특정 사용자에게 추천할 영화를 예측

In [25]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.dataset import DatasetAutoFolds
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# CSV 파일 경로 설정
ratings_file_path = '/content/drive/MyDrive/kdt_240424/m5_머신러닝/dataset/ml-latest-small/ratings.csv'
movies_file_path = '/content/drive/MyDrive/kdt_240424/m5_머신러닝/dataset/ml-latest-small/movies.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
ratings = pd.read_csv(ratings_file_path)
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [27]:
movies = pd.read_csv(movies_file_path)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [28]:
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5.0))
data_folds = DatasetAutoFolds(ratings_file='/content/drive/MyDrive/kdt_240424/m5_머신러닝/dataset/ml-latest-small/ratings_noh.csv', reader=reader)

In [29]:
# 전체 데이터를 학습 데이터로 생성
trainset = data_folds.build_full_trainset()
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [30]:
# movies 데이터 로드
movies = pd.read_csv(movies_file_path)

In [31]:
# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인
movieIds = ratings[ratings['userId'] == 9]['movieId']

if movieIds[movieIds == 42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId'] == 42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


- algo.predict() 메서드를 사용하여 특정 사용자(uid=9)와 특정 영화(iid=42)에 대한 평점을 예측합니다.
- verbose=True를 설정하면 예측 결과를 자세히 출력합니다.
- pred 변수는 예측 결과를 담고 있는 Prediction 객체입니다. 이 객체에는 사용자 ID, 영화 ID, 실제 평점(없을 경우 None), 예측 평점, 기타 정보 등이 포함되어 있습니다.

In [32]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)
print(ratings[ratings['userId'] == 9]['movieId'].tolist())

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}
[41, 187, 223, 371, 627, 922, 923, 1037, 1095, 1198, 1270, 1674, 1987, 2011, 2012, 2023, 2300, 2877, 2901, 3173, 3328, 3735, 4131, 4558, 4993, 5218, 5378, 5445, 5447, 5451, 5481, 5507, 5841, 5843, 5872, 5890, 5891, 5893, 5902, 5952, 5956, 5962, 5965, 5988, 6001, 6044]


In [34]:
# 사용자가 보지 않은 영화 목록 생성 함수
def get_unseen_surprise(ratings, movies, userId):
    # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

    # 모든 영화들의 movieId를 리스트로 생성
    total_movies = movies['movieId'].tolist()

    # 모든 영화들의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:', len(seen_movies), '추천대상 영화수:', len(unseen_movies), '전체 영화수:', len(total_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [35]:
# 영화 추천 함수
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함
    def sortkey_est(pred):
        return pred.est

    # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]

    # top_n으로 추출된 영화의 정보 추출: 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122657
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


전체 코드

In [22]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.dataset import DatasetAutoFolds
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# CSV 파일 경로 설정
ratings_file_path = '/content/drive/MyDrive/kdt_240424/m5_머신러닝/dataset/ml-latest-small/ratings.csv'
movies_file_path = '/content/drive/MyDrive/kdt_240424/m5_머신러닝/dataset/ml-latest-small/movies.csv'

# ratings 데이터 로드
ratings = pd.read_csv(ratings_file_path)

# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5.0))
data_folds = DatasetAutoFolds(ratings_file='/content/drive/MyDrive/kdt_240424/m5_머신러닝/dataset/ml-latest-small/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성
trainset = data_folds.build_full_trainset()
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

# movies 데이터 로드
movies = pd.read_csv(movies_file_path)

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인
movieIds = ratings[ratings['userId'] == 9]['movieId']

if movieIds[movieIds == 42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId'] == 42])

uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)
print(ratings[ratings['userId'] == 9]['movieId'].tolist())

# 사용자가 보지 않은 영화 목록 생성 함수
def get_unseen_surprise(ratings, movies, userId):
    # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

    # 모든 영화들의 movieId를 리스트로 생성
    total_movies = movies['movieId'].tolist()

    # 모든 영화들의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:', len(seen_movies), '추천대상 영화수:', len(unseen_movies), '전체 영화수:', len(total_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

# 영화 추천 함수
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함
    def sortkey_est(pred):
        return pred.est

    # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]

    # top_n으로 추출된 영화의 정보 추출: 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama
user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}
[41, 187, 223, 371, 627, 922, 923, 1037, 1095, 1198, 1270, 1674, 1987, 2011, 2012, 2023, 2300, 2877, 2901, 3173, 3328, 3735, 4131, 4558, 4993, 5218, 5378, 5445, 5447, 5451, 5481, 5507, 5841, 5843, 5872, 5890, 5891, 5893, 5902, 5952, 5956, 5962, 5965, 5988, 6001, 6044]
평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named D